In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
# 1. DEFINIR PARÂMETROS
# ---------------------
IMG_SIZE = (160, 160)
BATCH_SIZE = 32
EPOCHS = 10

In [ ]:
# 2. CARREGAR DATASET DIRETAMENTE DA INTERNET (TFDS)
# ----------------------------------------------------
# O TFDS vai baixar e preparar o dataset 'tf_flowers' para nós.
# Dividimos o conjunto de treino em 80% para treino e 20% para validação.
(train_ds, validation_ds), ds_info = tfds.load(
    'tf_flowers',
    split=['train[:80%]', 'train[80%:]'], # Cria as divisões de treino e validação
    with_info=True,
    as_supervised=True, # Retorna os dados como tuplas (imagem, rótulo)
)

# Obter informações do dataset
NUM_CLASSES = ds_info.features['label'].num_classes
class_names = ds_info.features['label'].names
num_train_examples = ds_info.splits['train[:80%]'].num_examples
print(f"Número de classes: {NUM_CLASSES}")
print(f"Nomes das classes: {class_names}")
print(f"Número de imagens de treino: {num_train_examples}")


In [ ]:
# 3. PRÉ-PROCESSAR OS DADOS
# As imagens do dataset vêm em tamanhos variados. Precisamos redimensioná-las.
def format_image(image, label):
    image = tf.image.resize(image, IMG_SIZE)
    return image, label

# Aplicar o redimensionamento e otimizar o carregamento
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_ds.map(format_image, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.cache().shuffle(1000).batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)

validation_dataset = validation_ds.map(format_image, num_parallel_calls=AUTOTUNE)
validation_dataset = validation_dataset.batch(BATCH_SIZE).cache().prefetch(buffer_size=AUTOTUNE)


In [ ]:
# 4. CRIAR O MODELO BASE

base_model = tf.keras.applications.MobileNetV2(
    input_shape=(160, 160, 3),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False


In [ ]:

# 5. ADICIONAR CAMADA DE CLASSIFICAÇÃO

data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip('horizontal'),
    tf.keras.layers.RandomRotation(0.2),
])

inputs = tf.keras.Input(shape=(160, 160, 3))
x = data_augmentation(inputs)
x = tf.keras.applications.mobilenet_v2.preprocess_input(x)
x = base_model(x, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = tf.keras.layers.Dense(NUM_CLASSES)(x) # Usamos a variável NUM_CLASSES
model = tf.keras.Model(inputs, outputs)

In [ ]:
# 6. COMPILAR O MODELO

base_learning_rate = 0.0001
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

print("\nResumo do modelo:")
model.summary()


In [ ]:
# 7. TREINAR O MODELO

history = model.fit(
    train_dataset,
    epochs=EPOCHS,
    validation_data=validation_dataset
)


In [ ]:

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Acurácia de Treinamento')
plt.plot(val_acc, label='Acurácia de Validação')
plt.legend(loc='lower right')
plt.ylabel('Acurácia')
plt.ylim([min(plt.ylim()),1])
plt.title('Acurácia de Treinamento e Validação')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Perda de Treinamento')
plt.plot(val_loss, label='Perda de Validação')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.title('Perda de Treinamento e Validação')
plt.xlabel('época')
plt.show()

In [ ]:
# DESCONGELAR O MODELO BASE
base_model.trainable = True

# Vamos descongelar a partir da camada 100 (as primeiras continuam congeladas)
fine_tune_at = 100
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer = tf.keras.optimizers.RMSprop(learning_rate=base_learning_rate/10),
    metrics=['accuracy']
)

print("\nIniciando Fine-Tuning...")

# Continuar o treinamento por mais algumas épocas
fine_tune_epochs = 2
total_epochs =  EPOCHS + fine_tune_epochs

history_fine = model.fit(
    train_dataset,
    epochs=total_epochs,
    initial_epoch=history.epoch[-1],
    validation_data=validation_dataset
)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

image_url = "https://blog.giulianaflores.com.br/wp-content/uploads/2013/06/Tulipas-coloridas.jpg"
image_name = image_url.split('/')[-1]

# Baixa a imagem e salva localmente no Colab
image_path = tf.keras.utils.get_file(image_name, origin=image_url)

print(f"Imagem de teste baixada em: {image_path}")

# --- CARREGAR E PRÉ-PROCESSAR A IMAGEM ---

# Carrega a imagem do disco, redimensionando para o tamanho que o modelo espera (160x160)
img = tf.keras.utils.load_img(image_path, target_size=IMG_SIZE)
img_array = tf.keras.utils.img_to_array(img)
img_batch = tf.expand_dims(img_array, 0)


# --- FAZER A PREVISÃO ---

# Usa o modelo treinado para prever a classe da imagem
predictions = model.predict(img_batch)

# O resultado 'predictions' são logits (números brutos).
# Aplicamos a função Softmax para converter esses números em probabilidades (de 0 a 1)
score = tf.nn.softmax(predictions[0])


# --- EXIBIR O RESULTADO ---

# Pega o nome da classe com a maior probabilidade
predicted_class = class_names[np.argmax(score)]
# Pega o valor da maior probabilidade
confidence = 100 * np.max(score)

# Imprime o resultado formatado
print(f"\nO modelo classifica esta imagem como: '{predicted_class}'")
print(f"Confiança: {confidence:.2f}%")

# Exibe a imagem testada com seu título de previsão
plt.figure(figsize=(6,6))
plt.imshow(img)
plt.title(f"Previsão: {predicted_class} ({confidence:.2f}%)")
plt.axis("off")
plt.show()